In [7]:
import tensorflow as tf
import os
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
import nengo
import nengo_dl
# https://colab.research.google.com/github/nengo/nengo-dl/blob/master/docs/examples/keras-to-snn.ipynb#scrollTo=ICv7sPqw6hO6

In [8]:
batch_size = 128
epochs = 10
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
class_count = 10

def reshape_for_snn(data): return data.reshape(data.shape[0], 1, -1)

x_train, y_train = reshape_for_snn(x_train), reshape_for_snn(y_train)
x_test, y_test = reshape_for_snn(x_test), reshape_for_snn(y_test)


input_layer = Input(shape=(28, 28, 1))
conv1 = Conv2D(filters=32, kernel_size=3, activation='relu')(input_layer)
pooling = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(filters=64, kernel_size=3, strides=2, activation='relu')(pooling)
flatten = Flatten()(conv2)
dense = Dense(128, activation='relu')(flatten)
dropout = Dropout(.2)(dense)
output_layer = Dense(class_count)(dropout)

tensorflow_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)


converter = nengo_dl.Converter(tensorflow_model)
with nengo_dl.Simulator(converter.net, minibatch_size=batch_size) as simulator:
    simulator.compile(optimizer=tf.optimizers.RMSprop(.001), loss='categorical_crossentropy', metrics=['accuracy'])
    simulator.fit(
        { converter.inputs[input_layer]: x_train },
        { converter.outputs[output_layer]: y_train },
        validation_data=(
            { converter.inputs[input_layer]: x_test },
            { converter.outputs[output_layer]: y_test }
        ),
        epochs=epochs
    )

    # save the parameters to file
    params_file = os.path.join('..', 'snn-params', 'keras_snn_params')
    simulator.save_params(params_file)

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/10


c:\users\itznu\pycharmprojects\use-of-snn\venv\lib\site-packages\nengo_dl\converter.py:309: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
c:\users\itznu\pycharmprojects\use-of-snn\venv\lib\site-packages\nengo_dl\converter.py:309: UserWarning: Layer type <class 'tensorflow.python.keras.layers.core.Dropout'> does not have a registered converter. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
c:\users\itznu\pycharmprojects\use-of-snn\venv\lib\site-packages\nengo_dl\simulator.py:467: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  "No GPU support detected. See "
c:\users\itznu\pycharmprojects\use-of-snn\venv\lib\site-packages\nengo_dl\simulator.py:1932: UserWarning: Number of 

ValueError: in user code:

    c:\users\itznu\pycharmprojects\use-of-snn\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\users\itznu\pycharmprojects\use-of-snn\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\itznu\pycharmprojects\use-of-snn\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\itznu\pycharmprojects\use-of-snn\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\itznu\pycharmprojects\use-of-snn\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs

In [ ]:
# run network function

activation = nengo.RectifiedLinear()
steps = 10
scale_firing_rates = 1
synapse = None
number_test = 400

nengo_converter = nengo_dl.Converter(tensorflow_model, swap_activations={ tf.nn.relu: activation },
        scale_firing_rates = scale_firing_rates, synapse=synapse)

nengo_input = nengo_converter.inputs[input_layer]
nengo_output = nengo_converter.outputs[output_layer]

sample_neurons_conv1 = np.linspace(0, np.prod(conv1.shape[1:]), 1000, endpoint=False, dtype=np.int32)
sample_neurons_conv2 = np.linspace(0, np.prod(conv2.shape[1:]), 1000, endpoint=False, dtype=np.int32)

with nengo_converter.net:
    conv1_probe = nengo.Probe(nengo_converter.layers[conv1][sample_neurons_conv1])
    conv2_probe = nengo.Probe(nengo_converter.layers[conv2][sample_neurons_conv2])

tiled_x_test = np.tile(x_test[:number_test], (1, steps, 1))

with nengo_converter.net:
    nengo_dl.configure_settings(stateful=False)

with nengo_dl.Simulator(nengo_converter.net, minibatch_size=10, progress_bar=False) as nengo_simulator:
    nengo_simulator.load_params(params_file)
    data = nengo_simulator.predict({ nengo_input: tiled_x_test })

predictions = np.argmax(data[nengo_output][:, -1], axis=-1)
accuracy = (predictions == y_test[:number_test, 0, 0]).mean()
print("Test accuracy: %.2f%%" % (100 * accuracy))